# This notebook demonstrates the admin-only features

In [ ]:
from meganno_client import Service, Authentication
import pprint
import pandas as pd

# Admin-only features

In [ ]:
project_name = 'meganno'
project_host = 'http://localhost' 
project_port = 5000 
my_token=<owner_distributed_token>

In [ ]:
auth = Authentication(project=project_name,
                      host=project_host,
                      port=project_port,
                      token=my_token)

In [ ]:
demo = Service(project=project_name,
               host=project_host,
               port=project_port,
               auth=auth)

# Updating schema

In [ ]:
demo.get_schemas().set_schemas({
    'label_schema': [
        {
            "name": "sentiment",
            "level": "record", 
            "options": [
                { "value": "pos", "text": "positive" },
                { "value": "neg", "text": "negative" },
                { "value": "neu", "text": "neutral" },
                
            ]
        },
        # adding a span-level label
                {
            "name": "sp",
            "level": "span", 
            "options": [
                { "value": "pos", "text": "positive" },
                { "value": "neg", "text": "negative" },
            ]
        }
    ]
})
demo.get_schemas().value(active=True)

In [ ]:
# get schema history
demo.get_schemas().get_history()

In [ ]:
demo.get_schemas().value(active=False)

# Loading data

In [ ]:
# import from url
# demo.import_data_url(url, column_mapping)

df = pd.read_csv("tweets.csv").loc[:300]
demo.import_data_df(df, column_mapping={
    'id':'id',
    'content':'content',
    "metadata":'pseudo_label' # optional metadata
})
# Note: In order to import a new dataset, we recommend to do so within a new project environment.

# Utilizing metadata 

## Example 1 : Customized Embedding

In [ ]:
#!pip install sentence-transformers

In [ ]:
# Example 1, adding sentence-bert embedding.
# metadata searches.

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# set metadata generation function 
demo.set_metadata("bert-embedding",lambda x: list(model.encode(x).astype(float)), 500)

In [ ]:
s3 = demo.search(keyword='delay', limit=1)
s3.show()

In [ ]:
s3.value()

In [ ]:
s4 = s3.suggest_similar(record_meta_name='bert-embedding'
                          , limit=4)
s4.show()

## Example 2 : Showing Hashtags As Context

In [ ]:
# user defined function
def extract_hashtags(text):
    hashtag_list = []
    for word in text.split():
        if word[0] == '#':
            hashtag_list.append(word[:])
    # widget can render markdown text
    return ''.join(['- {}\n'.format(x) for x in hashtag_list])

# apply metadata to project
demo.set_metadata("hashtag",lambda x: extract_hashtags(x), 500)

In [ ]:
s1= demo.search(keyword='',
              limit=50)
s1.show({'view':'table',
        'record_meta_names':['hashtag']})

# Exploration

In [ ]:
s1 = demo.search(regex='.* (delay) .*', limit=50, skip=0)
s1.show({'view':'table'})

In [ ]:
s2 = demo.search(keyword='good', limit=50, skip=0)
s2.show({'view':'table'})

In [ ]:
#subset computation:
# intersection
s_intersection = s1 & s2 # or s1.intersection(s2)
# union
s_union = s1 | s2 # or s1.union(s2)
# difference
s_diff = s1 - s2 # or s1.difference(s2)

# Analysis Dashboard

In [ ]:
#Analysis 
demo.show()

In [ ]:
demo.export()

# Assignment

In [ ]:
# assign subset to a user
# they can retrieve by `<user_service>.get_assignment()`
s_union.assign(<annotator_id>)